In [217]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from time import sleep
import sqlite3 as lite
import pandas as pd

# Web Scraping with Selenium, BeautifulSoup

In [243]:
def scraper(city_input):
    city = city_input[:-12]
    result_list = []
    for i in range(1, 30+1):
        soup = bs(ch_driver.page_source, "html.parser")
        results = soup.find('div', attrs={'id':'EATERY_SEARCH_RESULTS'}) 
        rank = str(i)
        restaurant = results.find('div', attrs={'data-index': '{rank}'.format(rank = rank)})

        title_div = restaurant.find('div', attrs={'class' : 'title'})
        title = title_div.find('a').text.strip('\n')

        rating_div = restaurant.find('div', attrs={'class' : 'rating rebrand'})
        rating = rating_div.find('span').attrs['alt']

        count = restaurant.find('span', attrs={'class' : 'reviewCount'}).find('a').text.strip('\n')

        try:
            price = restaurant.find('span', attrs={'class' : 'item price'}).text
            cuisine_div = restaurant.find('div', attrs={'class' : 'cuisines'}).find_all('a')
            cuisine_list = []
            for div in cuisine_div:
                cuisine_list.append(div.text)
            cuisine = ", ".join(cuisine_list)
        except:
            price = "N/A"
            cuisine = "N/A"

        review_ul = restaurant.find('ul', attrs={'class' : 'review_stubs review_snippets rebrand'})
        review_a = review_ul.find_all('a')
        review_list = []
        for tag in review_a:
            review_list.append(tag.text)
        review = ", ".join(review_list)

        result_list.append((city, rank, title, rating, count, price, cuisine, review))
        
    return result_list

In [244]:
options = webdriver.ChromeOptions()
options.add_argument('headless')

ch_driver = webdriver.Chrome('chromedriver_path', chrome_options=options)

url = 'https://www.tripadvisor.com/Restaurants-g191-United_States.html'
ch_driver.get(url)
ch_driver.implicitly_wait(3)

#ch_driver.find_element_by_class_name('continue').click()
#sleep(3)

cities_div = ch_driver.find_elements_by_class_name('geo_name')
city_len = len(cities_div)

all_result_list = []

for j in range(city_len):
    cities_div = ch_driver.find_elements_by_class_name('geo_name')

    city_button = cities_div[j].find_element_by_tag_name('a')
    city_name = city_button.text

    city_button.click()
    sleep(3)
    
    result = scraper(city_name)
    #print(result)
    all_result_list.extend(result)
    sleep(3)
    
    ch_driver.back()
    sleep(3)

ch_driver.quit()

c:\python37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


In [247]:
len(all_result_list)

600

In [248]:
# for check
all_result_list[350]

('Portland',
 '21',
 'Lechon',
 '4.5 of 5 bubbles',
 '208 reviews ',
 '$$ - $$$',
 'Latin, Argentinean, South American, Vegetarian Friendly, Vegan Options, Gluten Free Options',
 'A must in portland, A different spin on Argenitean food')

# Creating a DB with SQLite

`.open restaurants.db`

In [249]:
db = lite.connect('./restaurants.db')

In [250]:
pd.options.display.max_colwidth = 300

In [253]:
query_create_table = """
CREATE TABLE IF NOT EXISTS
TripAdvisor
("City" TEXT,
"Rank" TEXT,
"Restaurant Name" TEXT,
"Rating" TEXT,
"Review Count" TEXT,
"Price" TEXT,
"Cuisine" TEXT,
"Review" TEXT);
"""

query_insert_data = """
INSERT INTO TripAdvisor VALUES(?, ?, ?, ?, ?, ?, ?, ?);
"""

with db:
    cur = db.cursor()
    cur.execute(query_create_table)
    for row in all_result_list:
        cur.execute(query_insert_data, row)

In [256]:
query = """
SELECT * FROM TripAdvisor;
"""

pd.read_sql(query, db).head()

,City,Rank,Restaurant Name,Rating,Review Count,Price,Cuisine,Review
0,New York City,1,Piccola Cucina Osteria,4.5 of 5 bubbles,"1,859 reviews",$$ - $$$,"Italian, Mediterranean, Vegetarian Friendly, Vegan Options, Gluten Free Options","Amazing dinner, Amazing night"
1,New York City,2,Levain Bakery,4.5 of 5 bubbles,"4,419 reviews",N/A,N/A,"If you like cookies don’t miss this locati..., Warm and Delicious - Gooey Cookie!"
2,New York City,3,Levain Bakery,5 of 5 bubbles,"1,562 reviews",$$ - $$$,"American, Vegetarian Friendly","Heaven!, Delicious"
3,New York City,4,Piccola Cucina,4.5 of 5 bubbles,663 reviews,$$ - $$$,"Italian, Vegetarian Friendly, Vegan Options, Gluten Free Options","Best lazagna!!!, Wonderful birthday dinnner"
4,New York City,5,SottoCasa Pizzeria,5 of 5 bubbles,303 reviews,$$ - $$$,"Pizza, Vegetarian Friendly, Vegan Options, Gluten Free Options","Pizza, Absolutely the best pizza in New York"
